In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import seaborn as sns
import re
import sys

In [ ]:
sns.set_context('paper')
sns.set_color_codes()
sns.set_style('ticks')

In [ ]:
sys.path.append('../../../ar40-aug15/notebooks/')

In [ ]:
import cmaps

In [ ]:
def parse_output(fp):
    times = []
    perfs = []
    with open(fp) as f:
        for l in f:
            m = re.match(r'N=(\d+) TH=(\d+)', l)
            if m:
                N = m.group(1)
                num_threads = m.group(2)

            m = re.match(r'Seconds: (\d\.\d+e[\+,\-]\d+)\s*(\d\.\d+e[\+,\-]\d+)\s*(\d\.\d+e[\+,\-]\d+)', l)
            if m:
                times.append([float(x) for x in (N, num_threads, m.group(1), m.group(2), m.group(3))])

            m = re.match(r'FLOPS: (\d\.\d+e[\+,\-]\d+)\s*(\d\.\d+e[\+,\-]\d+)\s*(\d\.\d+e[\+,\-]\d+)', l)
            if m:
                perfs.append([float(x) for x in (N, num_threads, m.group(1), m.group(2), m.group(3))])

    timesdf = pd.DataFrame(times, columns=('N', 'num_threads', 'min_time', 'mean_time', 'max_time'))
    perfsdf = pd.DataFrame(perfs, columns=('N', 'num_threads', 'min_perf', 'mean_perf', 'max_perf'))
    return pd.merge(timesdf, perfsdf, on=('N', 'num_threads'))

In [ ]:
macres = parse_output('../outputs/mac_output.txt')
hpccres = parse_output('../outputs/hpcc-results.txt')

In [ ]:
def model(xs, n):
    return (2/2.5e9 + 1.6e-9)*xs**3 / max(n, 1)

xs = np.logspace(1, 4, 100)

plt.figure(figsize=(4, 2.8))

d = hpccres[hpccres.num_threads == 0]
plt.plot(d.N, d.min_time, 'bo', label='No OpenMP (Data)')
plt.plot(xs, model(xs, 0), 'b-', label='No OpenMP (Model)')

d = hpccres[hpccres.num_threads == 4]
plt.plot(d.N, d.min_time, 'rs', label='4 threads (Data)')
plt.plot(xs, model(xs, 4), 'r--', label='4 threads (Model)')

d = hpccres[hpccres.num_threads == 20]
plt.plot(d.N, d.min_time, 'g^', label='20 threads (Data)')
plt.plot(xs, model(xs, 20), 'g-.', label='20 threads (Model)')

plt.loglog()
plt.legend(loc=2)
sns.despine()
plt.xlabel('Dimension of matrix')
plt.ylabel('Time [s]')

plt.text(0.9, 0.1, r'$t \sim (2c+r)N_{pts}^3 / N_{threads}$', 
         transform=plt.gca().transAxes, ha='right', va='bottom')
plt.savefig('dim_v_time.pdf', bbox_inches='tight')

In [ ]:
with sns.axes_style('ticks'):
    d1 = hpccres[hpccres.N == 20]
    d2 = hpccres[hpccres.N == 100]
    d3 = hpccres[hpccres.N == 2000]

    fig, ax = plt.subplots(3, 1, squeeze=True, figsize=(3, 5.5), sharex=True)

    for a in ax:
        a.ticklabel_format(format='sci', scilimits=(0, 0), axis='y')
#         a.grid(axis='y')
        a.set_ylabel('Time [s]')

    ax[0].plot(d1.num_threads, d1.min_time, 'o-')
    ax[0].set_title("N = 20")
    ax[1].plot(d2.num_threads, d2.min_time, 'o-')
    ax[1].set_title("N = 100")
    ax[2].plot(d3.num_threads, d3.min_time, 'o-')
    ax[2].set_title("N = 2000")

    ax[2].set_xlabel('Number of threads')

    plt.xlim(-1, 21)
    sns.despine()
    
    plt.tight_layout()
    plt.savefig('th_v_time.pdf')

In [ ]:
sns.set_context('notebook')

In [ ]:
with sns.axes_style('white'):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    m = ax.plot_trisurf(hpccres.N, hpccres.num_threads, np.log10(hpccres.min_time), cmap=cmaps.viridis)
    ax.view_init(30, 140)
    ax.set_xlabel('Dimension of matrix')
    ax.set_ylabel('Number of threads')
    ax.set_zlabel('log(time)')
    plt.tight_layout()
    plt.savefig('3d.pdf', bbox_inches='tight')